### Figure S1b

In [ ]:
library(maftools)
library(ComplexHeatmap)
library(dplyr)
library(circlize)
library(gridExtra)

options(stringsAsFactors = F)

In [ ]:
HRR_genes <- c("ATM", "BRCA1", "BRCA2", "BARD1", "BRIP1", "CDK12", "CHEK1", 
                "CHEK2", "FANCL", "PALB2", "RAD51B", "RAD51C", "RAD51D", "RAD54L", "PPP2R2A")

In [ ]:
germline_laml = read.maf(maf = './OCEC_germline_maf.maf')

germline_maf_data <- subsetMaf(maf = germline_laml, genes = HRR_genes, isTCGA = FALSE)
all_samples <- unique(germline_laml@data$Tumor_Sample_Barcode)


In [ ]:
germline_mutations <- germline_maf_data@data[, c("Hugo_Symbol", "Tumor_Sample_Barcode","Variant_Classification")]
germline_mutations$Hugo_Symbol <- as.character(germline_mutations$Hugo_Symbol)
germline_mutations$Tumor_Sample_Barcode <- as.character(germline_mutations$Tumor_Sample_Barcode)
germline_mutations$Variant_Classification <- as.character(germline_mutations$Variant_Classification)


In [ ]:
germline_mut_matrix <- matrix("", nrow = length(HRR_genes), ncol = length(all_samples),
                     dimnames = list(HRR_genes, all_samples))

for (i in 1:nrow(germline_mutations)) {
    gene <- germline_mutations$Hugo_Symbol[i]
    sample <- germline_mutations$Tumor_Sample_Barcode[i]

    germline_mut_matrix[gene, sample] <- "Germline mutation"
}

In [ ]:
laml = read.maf(maf = '/OCEC_somatic_maf.maf')
laml_mut <- getGeneSummary(laml)

genes <- HRR_genes
maf_data <- subsetMaf(maf = laml, genes = genes, isTCGA = FALSE)
all_samples <- unique(laml@data$Tumor_Sample_Barcode)
mutations <- maf_data@data[, c("Hugo_Symbol", "Tumor_Sample_Barcode","Variant_Classification")]
mutations$Hugo_Symbol <- as.character(mutations$Hugo_Symbol)
mutations$Tumor_Sample_Barcode <- as.character(mutations$Tumor_Sample_Barcode)
mutations$Variant_Classification <- as.character(mutations$Variant_Classification)
order <- c('Multi_Hit','Missense_Mutation','Nonsense_Mutation','Frame_Shift_Del','Frame_Shift_Ins','In_Frame_Del','In_Frame_Ins','Splice_Site')
laml@data$Variant_Classification <- factor(laml@data$Variant_Classification, levels = order)

somatic_mut_matrix <- matrix("", nrow = length(genes), ncol = length(all_samples),
                     dimnames = list(genes, all_samples))

for (i in 1:nrow(mutations)) {
    gene <- mutations$Hugo_Symbol[i]
    sample <- mutations$Tumor_Sample_Barcode[i]

    somatic_mut_matrix[gene, sample] <- "Somatic mutation"

}


In [ ]:
both_colnames <- intersect(colnames(germline_mut_matrix), colnames(somatic_mut_matrix))
germline_mut_matrix <- germline_mut_matrix[,both_colnames]
somatic_mut_matrix <- somatic_mut_matrix[,both_colnames]

germline_mut_df <- as.data.frame(germline_mut_matrix)
somatic_mut_df <- as.data.frame(somatic_mut_matrix)

merge_samples <- function(x, y, z) {
  x <- ifelse(is.na(x), "", paste(x, ";", sep = ""))
  y <- ifelse(is.na(y), "", paste(y, ";", sep = ""))

  combined <- paste(x, y, sep = "")
  combined <- gsub(";;+", ";", combined)
  
  combined <- gsub("^(;+)", "", combined)
  
  combined
}

HRR_df <- mapply(FUN = merge_samples, germline_mut_df, somatic_mut_df, SIMPLIFY = FALSE)
HRR_df <- data.frame(HRR_df, row.names = rownames(germline_mut_df))

alter_samples_index <- apply(HRR_df, 2, function(col) {
    !all(col == "")
})
HRR_alter_df <- HRR_df[, alter_samples_index]
HRR_matrix <- as.matrix(HRR_alter_df)

In [ ]:
clinical_data <- read.csv("./germline_clinical_df.csv",row.names=1)
clinical_data$TP53 <- as.character(clinical_data$TP53)
clinical_data$PTEN <- as.character(clinical_data$PTEN)
clinical_data$ARID1A <- as.character(clinical_data$ARID1A)
clinical_data$PIK3CA <- as.character(clinical_data$PIK3CA)

clinical_data <- clinical_data[colnames(HRR_matrix),]
clinical_data$Tumor_Type <- factor(clinical_data$Tumor_Type, levels=c("EC","OC"))
clinical_data$Tumor_SubType <- factor(clinical_data$Tumor_SubType, levels=c("Serous","Endometrial","Mucinous","Clear cell","Sarcoma","Others"))
clinical_data <- clinical_data[order(clinical_data$Tumor_Type, clinical_data$Tumor_SubType), ]
HRR_matrix <- HRR_matrix[,rownames(clinical_data)]


In [ ]:
ha <- HeatmapAnnotation(
  Tumor_Type = clinical_data$Tumor_Type,
  Subtype = clinical_data$Tumor_SubType,
  Recurrence = clinical_data$Recurrence_Status,
  MSI = clinical_data$MSI_state,
  annotation_height = unit(8, "mm"),
  show_annotation_name = TRUE,
  annotation_name_side = "left",
  col = list(
    Tumor_Type = c("OC" = "#ab526a", "EC" = "#d68081", 'Unknown' = "#dcdcdc"),
    Subtype = c("Mucinous" = "#b1bcf4", "Serous" = "#65a56d", "Clear cell" = "#f4e88d", "Endometrial" = "#eab768", "Sarcoma" = "#cf5b5d",'Others' = "#dcdcdc"),
    Recurrence = c("1" = "#785190", "0" = "#6f87bb", 'Unknown' = "#dcdcdc"),
    MSI = c("MSI-low" = "#62b58f", "MSI-high" = "#e7cc5e", 'Unknown' = "#dcdcdc")
  ),
  annotation_legend_param = list(
    Tumor_Type = list(border = TRUE, border_gp = gpar(col = "white")),
    Subtype = list(border = TRUE, border_gp = gpar(col = "white")),
    Recurrence = list(border = TRUE, border_gp = gpar(col = "white")),
    MSI = list(border = TRUE, border_gp = gpar(col = "white"))
  )
)

In [ ]:
col = c("Germline mutation" = "#3e726e", "Somatic mutation" = "#f4c744")
alter_fun = list(
    background = function(x, y, w, h) {
        grid.rect(x, y, w*0.2, h-unit(2, "pt"), 
            gp = gpar(fill = "#CCCCCC", col = NA))
    },
    `Germline mutation` = function(x, y, w, h) {
        grid.rect(x, y, w*0.7, h-unit(2, "pt"), 
            gp = gpar(fill = col["Germline mutation"], col = "#f69ca0"))
    },
    `Somatic mutation` = function(x, y, w, h) {
        grid.rect(x, y, w*0.7, h-unit(2, "pt"), 
            gp = gpar(fill = col["Somatic mutation"], col = "#415eab"))
    }
)

column_title = "Alterations"
heatmap_legend_param = list(title = "Alternations", at = c("Germline mutation", "Somatic mutation"), 
        labels = c("Germline mutation", "Somatic mutation"))

HRR_plt <- oncoPrint(HRR_matrix,
    alter_fun = alter_fun, col = col,right_annotation = NULL,left_annotation = NULL,show_pct = FALSE, column_order = 1:ncol(BRCA_matrix),
    row_names_side = "left",top_annotation = ha, column_title = NULL, heatmap_legend_param = heatmap_legend_param)

print(HRR_plt)
